In [552]:
import json
from web3 import Web3
from web3.middleware import geth_poa_middleware
import pickle
import requests
import pandas as pd
from datetime import datetime
import numpy as np

In [553]:
web3 = Web3(Web3.HTTPProvider("https://bsc-dataseed.binance.org/"))
web3.middleware_onion.inject(geth_poa_middleware, layer=0)
web3.isConnected()

True

In [554]:
apikey = 'GKDPH6RRX5YSH1TYPMNHNVKGMB6I9V3I8T'

address = "0xF700D4c708C2be1463E355F337603183D20E0808"

In [555]:
#Get ABI

url = "https://api.bscscan.com/api"
params = {
    "module": "contract",
    "action": "getabi",
    "address": address,
    "apikey": apikey
}

response = requests.get(url, params=params)

if response.status_code == 200:
    abi = response.json()["result"]
    print("Abi recibida correctamente")
else:
    print("Error en la solicitud")

Abi recibida correctamente


In [556]:
w3address = Web3.toChecksumAddress(address.lower())
contract = web3.eth.contract(address=w3address, abi=abi)

token = contract.functions.name().call()
symbol = contract.functions.symbol().call()

print(f"Abi del token: {token}, con simbolo: {symbol}, recibida correctamente")

Abi del token: Galactic Quadrant, con simbolo: GQ, recibida correctamente


In [557]:
#Current block

currentblock = web3.eth.blockNumber
endblock = currentblock

In [558]:
#Lista de topics0

approval_method = '0x8c5be1e5ebec7d5bd14f71427d1e84f3dd0314c0f7b2291e5b200ac8c7c3b925'
transfer_method = '0xddf252ad1be2c89b69c2b068fc378daa952ba7f163c4a11628f55a4df523b3ef'
stake_method = '0xe1fffcc4923d04b559f4d29a8bfc6cda04eb5b0d3c460751c2402c5c5cc9109c'
address = '0xF700D4c708C2be1463E355F337603183D20E0808'

#Set block
currentblock = web3.eth.blockNumber
endblock = currentblock

#Set topic0
topic0 = approval_method

In [559]:
#Get "Internal Transactions" by Block Range

def get_internal_transactions(endblock, numberofblocks):
    url = "https://api.bscscan.com/api"
    params = {
        "module": "account",
        "action": "txlistinternal",
        "startblock": endblock - numberofblocks,
        "endblock": endblock,
        "page": 1,
        "offset": 10,
        "sort": "asc",
        "apikey": apikey
    }

    response = requests.get(url, params=params)

    if response.status_code == 200:
        transactions = response.json()["result"]
        return transactions
    else:
        return "Error en la solicitud"

#Get Logs by Address sin topic0

def get_logs_loop(address, endblock, blocks):
    if blocks % 1000 != 0:
        raise ValueError("El número de bloques debe ser múltiplo de 1000")
    total_logs = []
    for loop in range(int(blocks/1000)):
        logs = get_logs1000(address,endblock-loop*1000)
        total_logs = total_logs + logs
    return total_logs 


def get_logs1000(address, endblock):
    url = "https://api.bscscan.com/api"
    params = {
        "module": "logs",
        "action": "getLogs",
        "fromBlock": endblock - 1000,
        "toBlock": endblock,
        "address": address,
        "apikey": apikey
    }

    response = requests.get(url, params=params)

    if response.status_code == 200:
        logs = response.json()["result"]
        return logs
    else:
        return "Error en la solicitud"

#Get Logs by Address

def get_logstopic_loop(address, endblock, blocks, topic0):
    if blocks % 1000 != 0:
        raise ValueError("El número de bloques debe ser múltiplo de 1000")
    total_logs = []
    for loop in range(int(blocks/1000)):
        logs = get_logs1000(address,endblock-loop*1000)
        total_logs = total_logs + logs
    return total_logs 


def get_logs1000topic(address, endblock, topic0):
    url = "https://api.bscscan.com/api"
    params = {
        "module": "logs",
        "action": "getLogs",
        "fromBlock": endblock - 1000,
        "toBlock": endblock,
        "topic0": topic0,
        "address": address,
        "apikey": apikey
    }

    response = requests.get(url, params=params)

    if response.status_code == 200:
        logs = response.json()["result"]
        return logs
    else:
        return "Error en la solicitud"


#Get Logs by Address and From

def get_logstopic0_1_loop(address, endblock, blocks, topic0, topic1):
    if blocks % 1000 != 0:
        raise ValueError("El número de bloques debe ser múltiplo de 1000")
    total_logs = []
    for loop in range(int(blocks/1000)):
        logs = get_logs1000(address,endblock-loop*1000)
        total_logs = total_logs + logs
    return total_logs 


def get_logs1000topic0_1(address, endblock, topic0, topic1):
    url = "https://api.bscscan.com/api"
    params = {
        "module": "logs",
        "action": "getLogs",
        "fromBlock": endblock - 1000,
        "toBlock": endblock,
        "topic0": topic0,
        "topic0_1": "and",
        "topic1": topic1,                
        "address": address,
        "apikey": apikey
    }

    response = requests.get(url, params=params)

    if response.status_code == 200:
        logs = response.json()["result"]
        return logs
    else:
        return "Error en la solicitud"

#Get a list of 'BEP-20 Token Transfer Events' by wallet

def get_token_transfers(wallet, address):
    url = "https://api.bscscan.com/api"
    params = {
        "module": "account",
        "action": "tokentx",
        "contractaddress": address,
        "address": wallet,
        "page": 1,
        "offset": 1000,
        "startblock": 0,
        "endblock": 999999999,
        "sort": "asc",
        "apikey": apikey,
    }

    response = requests.get(url, params=params)

    if response.status_code == 200:
        transactions = response.json()["result"]
        return transactions
    else:
        return "Error in the request"


In [560]:
#Diferentes topics con su hash

topic_approve = "0x8c5be1e5ebec7d5bd14f71427d1e84f3dd0314c0f7b2291e5b200ac8c7c3b925"
topic_burner = "0x696de425f79f4a40bc6d2122ca50507f0efbeabbff86a84871b7196ab8ea8df7"
topic_transfer = "0xddf252ad1be2c89b69c2b068fc378daa952ba7f163c4a11628f55a4df523b3ef"

claim_contract = "0x739e6e290f9aa6ef3a929950aa47795b1d187229"
stake_contract = "0xdf5c4c1b98a4d045573ab2005d77e87daa58c853"

0xB88D09175676913d69924ACac17Ec69037f1551E

#Liquidity Pool

pcs = "0x00000000000000000000000072121d60b0e2f01c0fb7fe32ca24021b42165a40"

#Set block
currentblock = web3.eth.blockNumber
endblock = currentblock

#Set topic0
topic0 = approval_method

In [562]:
## Extractor de Events/topics0

total_logs = get_logs_loop(address,endblock,100000)


#Save hash in a list

total_hash = []
for i in range(len(total_logs)):
    event_hash = total_logs[i]['topics'][0]
    total_hash.append(event_hash)

unique_event_hash = list(set(total_hash))


#Cantidad de logs extraidos

print(f"Se han obtenido {len(total_logs)} logs, en los cuales hay {len(unique_event_hash)} event hash unicos.")
print(unique_event_hash)


#Guarda los nuevos event hash

logs_unique = []
for value in unique_event_hash:
    if value not in logs_unique:
        logs_unique.append(value)

print(f"Actualmente hay almacenados {len(logs_unique)} event hash unicos.")

Se han obtenido 14689 logs, en los cuales hay 3 event hash unicos.
['0x8c5be1e5ebec7d5bd14f71427d1e84f3dd0314c0f7b2291e5b200ac8c7c3b925', '0x9d228d69b5fdb8d273a2336f8fb8612d039631024ea9bf09c424a9503aa078f0', '0xddf252ad1be2c89b69c2b068fc378daa952ba7f163c4a11628f55a4df523b3ef']
Actualmente hay almacenados 3 event hash unicos.


In [564]:
logs_topic = get_logs1000topic(address, endblock, topic_transfer)
logs_topic

[{'address': '0xf700d4c708c2be1463e355f337603183d20e0808',
  'topics': ['0xddf252ad1be2c89b69c2b068fc378daa952ba7f163c4a11628f55a4df523b3ef',
   '0x00000000000000000000000072121d60b0e2f01c0fb7fe32ca24021b42165a40',
   '0x000000000000000000000000def171fe48cf0115b1d80b88dc8eab59176fee57'],
  'data': '0x000000000000000000000000000000000000000000000e407504fe09f01b6525',
  'blockNumber': '0x182d33d',
  'blockHash': '0x07e542bfa6ee68e0e1673f299a843b8bc919c3ca94ab284012b08df3a485b8b6',
  'timeStamp': '0x63dd2cc8',
  'gasPrice': '0x12a05f200',
  'gasUsed': '0x49447',
  'logIndex': '0xae',
  'transactionHash': '0x4e3c913659c04ef696f37f00c8ecd0db05cd3100f95cf72301d54407c2f72609',
  'transactionIndex': '0x3d'},
 {'address': '0xf700d4c708c2be1463e355f337603183d20e0808',
  'topics': ['0xddf252ad1be2c89b69c2b068fc378daa952ba7f163c4a11628f55a4df523b3ef',
   '0x000000000000000000000000def171fe48cf0115b1d80b88dc8eab59176fee57',
   '0x000000000000000000000000c590175e458b83680867afd273527ff58f74c02b'],
 

In [565]:
logs_topic0_1 = get_logs1000topic0_1(address, endblock, topic_transfer, pcs)
logs_topic0_1

[{'address': '0xf700d4c708c2be1463e355f337603183d20e0808',
  'topics': ['0xddf252ad1be2c89b69c2b068fc378daa952ba7f163c4a11628f55a4df523b3ef',
   '0x00000000000000000000000072121d60b0e2f01c0fb7fe32ca24021b42165a40',
   '0x000000000000000000000000def171fe48cf0115b1d80b88dc8eab59176fee57'],
  'data': '0x000000000000000000000000000000000000000000000e407504fe09f01b6525',
  'blockNumber': '0x182d33d',
  'blockHash': '0x07e542bfa6ee68e0e1673f299a843b8bc919c3ca94ab284012b08df3a485b8b6',
  'timeStamp': '0x63dd2cc8',
  'gasPrice': '0x12a05f200',
  'gasUsed': '0x49447',
  'logIndex': '0xae',
  'transactionHash': '0x4e3c913659c04ef696f37f00c8ecd0db05cd3100f95cf72301d54407c2f72609',
  'transactionIndex': '0x3d'},
 {'address': '0xf700d4c708c2be1463e355f337603183d20e0808',
  'topics': ['0xddf252ad1be2c89b69c2b068fc378daa952ba7f163c4a11628f55a4df523b3ef',
   '0x00000000000000000000000072121d60b0e2f01c0fb7fe32ca24021b42165a40',
   '0x000000000000000000000000efbbd0e72f01f21710e065f8f2a3f20ff0fb235c'],
 

In [566]:
#Frame and clean api output

df = pd.DataFrame(logs_topic0_1)
df_clean = pd.concat([df['topics'], df['transactionHash']], axis=1)
df_topics = pd.DataFrame(df['topics'].tolist(), columns=['topic0', 'topic1', 'topic2'])
df_transfers = pd.concat([df_clean, df_topics], axis=1).drop(columns=['topics', 'topic0', 'topic1'], axis=1)
df_transfers.rename(columns={'topic2': 'To'}, inplace=True)
df_transfers['To'] = "0x" + df_transfers['To'].str[26:]
df_transfers

# #Filtros

# df_from_pcs = df_transfers[df_transfers['From'] == "0x00000000000000000000000072121d60b0e2f01c0fb7fe32ca24021b42165a40"]
# df_from_pcs.reset_index(drop=True, inplace=True)
# df_from_pcs

,transactionHash,To
0,0x4e3c913659c04ef696f37f00c8ecd0db05cd3100f95cf72301d54407c2f72609,0xdef171fe48cf0115b1d80b88dc8eab59176fee57
1,0x675936a6377ebe1a272525f3e7f6f4c16e9f855df0ae8b8014c240411090f5ea,0xefbbd0e72f01f21710e065f8f2a3f20ff0fb235c
2,0x711e7c3fff3518195df9ab0865270112dc10995a071b6d5628067b8b01722cd8,0x59cbd23225a0ae81e94d31d7d1e98a43d1899949
3,0x834ca4c7ea3c54ba83fdbbe7909f3a6ace9f433a7d7a1965c2d2ff678cc32e93,0xaf246bca652bf7f4816bf5f0502213a4e66e41c7
4,0xc28318d14dcfc6809c5e13bc045f07f7fcc877f78071737aba758fdaaa4c58e8,0xe3c998467f1fcfc0dbea62478f9ae2c209e45e4f
5,0x43eb6ef083ece07f29f6e3fcb70f3772e9928f952d5c4cb2a953cca1738a83da,0x5289c22f1ea776cc96bddfbf653629ebed72d02c
6,0x71986ac34da524e5df49287465c093faf8ee786837763aefa53fee43269b9e43,0x9353fecb8c6434d98bfcf1e8284f01c0a44affb1
7,0xf9bc1a5f9c155ec22cebf26c30e7c2c4cca6786a6122b68203984026077ed6bb,0xc03581c5177362bd8757c6844066840088f7b383
8,0x79a276216e53343189a4a4f84d450641f7a650c0cdbc7edbd54e6c6212556b66,0x09b032436a2ceb12d89e4f8a53e49f0ea5798eb6
9,0xc22a4c4d8dda87692aef824246db7b5d7f95cf281afee0773914e306c27abc3d,0x4f91e50566120e53309dff2e0c3fcbe36e2f3a4a


In [573]:
#Set Wallet
wallet = "0xdb5de412A2bF5eD7625288558bda49A39C20Bfe8"

#Get data
token_tx = get_token_transfers(wallet,address)

#Data Clean
token_tx = pd.DataFrame(token_tx)

token_tx['timeStamp'] = token_tx['timeStamp'].astype(int)
token_tx['timeStamp'] = token_tx['timeStamp'].apply(lambda x: datetime.fromtimestamp(x))

token_tx['mont'] = round(token_tx['value'].astype(float)/1000000000000000000, 2)

token_tx = token_tx.drop(columns=['blockNumber', 'nonce', 'contractAddress', 'tokenName', 'transactionIndex', 'gas', 'gasPrice', 'gasUsed', 'cumulativeGasUsed', 'input', 'confirmations', 'blockHash', 'value', 'tokenDecimal'], axis=1)

token_tx.rename(columns={'mont' : 'value'}, inplace=True)

token_tx.sort_values(by='timeStamp', ascending=True, inplace=True)

#Add balance 
token_tx['move_balance'] = np.where(token_tx['to'] == wallet.lower(), token_tx['value'], -token_tx['value'])
# balance = token_tx['move_balance'].sum()

#Staking Check & Balance
token_tx['staking'] = ((token_tx['to'] == stake_contract) | (token_tx['from'] == stake_contract)).astype(int)


token_tx['stake_balance'] = 0
for index, row in token_tx.iterrows():
    if row['staking'] == 1:
        if row['to'] == wallet.lower():
            token_tx.at[index, 'stake_balance'] = row['value']
        else:
            token_tx.at[index, 'stake_balance'] = -row['value']




In [574]:

#Balance Staked
print(f"Actualmente tiene {token_tx['stake_balance'].sum()} tokens stakeados")

#Tokens holds
print(f"y holdea {token_tx['move_balance'].sum()} tokens.")

Actualmente tiene 18256.119999999733 tokens stakeados
y holdea 314.04000000015367 tokens.


In [575]:
token_tx.head()

,timeStamp,hash,from,to,tokenSymbol,value,move_balance,staking,stake_balance
0,2022-08-17 17:34:51,0x87a77431ebc835b825988b18d17565b7e1baa114dc82a4d9a17a6dc18c30cddb,0x72121d60b0e2f01c0fb7fe32ca24021b42165a40,0xdb5de412a2bf5ed7625288558bda49a39c20bfe8,GQ,80690.37,80690.37,0,0.00
1,2022-08-18 17:53:07,0xe209c0cd15c80c026a92276bd6ef3901c3f365f3b379465f0b55d8b23c9f0dd0,0xdb5de412a2bf5ed7625288558bda49a39c20bfe8,0xdf5c4c1b98a4d045573ab2005d77e87daa58c853,GQ,80690.37,-80690.37,1,-80690.37
2,2022-08-31 14:27:18,0x938f4fe05038827f0013531fdc91bca8121615da8b98e187da6be6a33a41607d,0xdf5c4c1b98a4d045573ab2005d77e87daa58c853,0xdb5de412a2bf5ed7625288558bda49a39c20bfe8,GQ,135.62,135.62,1,135.62
3,2022-08-31 14:27:48,0x466b335eca3006b9a9b53f189095e5c8ca520754c4afa625ca47ad3db1b6b123,0xdf5c4c1b98a4d045573ab2005d77e87daa58c853,0xdb5de412a2bf5ed7625288558bda49a39c20bfe8,GQ,0.00,0.00,1,0.00
4,2022-08-31 14:27:48,0x466b335eca3006b9a9b53f189095e5c8ca520754c4afa625ca47ad3db1b6b123,0xdb5de412a2bf5ed7625288558bda49a39c20bfe8,0xdf5c4c1b98a4d045573ab2005d77e87daa58c853,GQ,135.62,-135.62,1,-135.62


In [572]:
token_tx

,timeStamp,hash,from,to,tokenSymbol,value,move_balance,staking,stake_balance
0,2022-08-17 17:34:51,0x87a77431ebc835b825988b18d17565b7e1baa114dc82a4d9a17a6dc18c30cddb,0x72121d60b0e2f01c0fb7fe32ca24021b42165a40,0xdb5de412a2bf5ed7625288558bda49a39c20bfe8,GQ,80690.37,-80690.37,0,0.00
1,2022-08-18 17:53:07,0xe209c0cd15c80c026a92276bd6ef3901c3f365f3b379465f0b55d8b23c9f0dd0,0xdb5de412a2bf5ed7625288558bda49a39c20bfe8,0xdf5c4c1b98a4d045573ab2005d77e87daa58c853,GQ,80690.37,-80690.37,1,-80690.37
2,2022-08-31 14:27:18,0x938f4fe05038827f0013531fdc91bca8121615da8b98e187da6be6a33a41607d,0xdf5c4c1b98a4d045573ab2005d77e87daa58c853,0xdb5de412a2bf5ed7625288558bda49a39c20bfe8,GQ,135.62,-135.62,1,135.62
3,2022-08-31 14:27:48,0x466b335eca3006b9a9b53f189095e5c8ca520754c4afa625ca47ad3db1b6b123,0xdf5c4c1b98a4d045573ab2005d77e87daa58c853,0xdb5de412a2bf5ed7625288558bda49a39c20bfe8,GQ,0.00,-0.00,1,0.00
4,2022-08-31 14:27:48,0x466b335eca3006b9a9b53f189095e5c8ca520754c4afa625ca47ad3db1b6b123,0xdb5de412a2bf5ed7625288558bda49a39c20bfe8,0xdf5c4c1b98a4d045573ab2005d77e87daa58c853,GQ,135.62,-135.62,1,-135.62
5,2022-09-01 16:54:47,0x148e9606449902fe24133b32088e13af42088c98b77ca0fc23fca940ee59588d,0x72121d60b0e2f01c0fb7fe32ca24021b42165a40,0xdb5de412a2bf5ed7625288558bda49a39c20bfe8,GQ,198570.04,-198570.04,0,0.00
6,2022-09-01 16:56:53,0x46bd24a027f3e514e515465c3dc0c040d4f9e2e46e2f1b579c8f7b808d4d8b2f,0xdf5c4c1b98a4d045573ab2005d77e87daa58c853,0xdb5de412a2bf5ed7625288558bda49a39c20bfe8,GQ,11.85,-11.85,1,11.85
7,2022-09-01 16:56:53,0x46bd24a027f3e514e515465c3dc0c040d4f9e2e46e2f1b579c8f7b808d4d8b2f,0xdb5de412a2bf5ed7625288558bda49a39c20bfe8,0xdf5c4c1b98a4d045573ab2005d77e87daa58c853,GQ,198570.04,-198570.04,1,-198570.04
8,2022-09-08 10:14:17,0x77f01cc31d0770b891be275762c8f4e8899861824e5d1f3b2472fc2e7c85259a,0x72121d60b0e2f01c0fb7fe32ca24021b42165a40,0xdb5de412a2bf5ed7625288558bda49a39c20bfe8,GQ,355542.31,-355542.31,0,0.00
9,2022-09-08 10:32:05,0x420b7963f61d06020765c410fa3d9fa279e38524b6f784928e5f706fffe32ee9,0xdf5c4c1b98a4d045573ab2005d77e87daa58c853,0xdb5de412a2bf5ed7625288558bda49a39c20bfe8,GQ,241.44,-241.44,1,241.44
